In [112]:
%pylab inline
import time
import re
import matplotlib.pyplot as plt
import pandas as pd
import string
import codecs
import os
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from sklearn import naive_bayes as bayes
from sklearn.model_selection import train_test_split
#簡轉繁
from hanziconv import HanziConv

#ham與spam的每封郵件list
ham_text = []
spam_text = []
#ham與spam的數據集
ham_data = []
spam_data = []
#取得目前工作目錄
SaveDirectory = os.getcwd() 
#獲得文檔列表
listham = os.listdir(SaveDirectory+'\\data\\normal')
listspam = os.listdir(SaveDirectory+'\\data\\spam')
#數據集數量

print('spam total：',len(listspam))
print('ham total：',len(listham))


Populating the interactive namespace from numpy and matplotlib
spam total： 7775
ham total： 7063


C:\Users\zxc98\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [113]:
for fileName in listham:
        with open('data/normal/'+fileName, 'r',encoding='gbk') as f:
            #文本每一列儲存
            text=[]
            # 過濾非中文字符，正規化
            for line in f.readlines():
                pattern = re.compile('[^\u4e00-\u9fa5]') #正規化(去除非中文字符)
                line = pattern.sub("", line) #將其他字符取代為""
                content = line.strip().split() #去除一些空白.換行
                text = text+content #儲存這個文本的內容
            text = " ".join(text) #將文本內容list加入全部文本的list
            ham_text.append(HanziConv.toTraditional(text)) #簡轉繁
ham_data = pd.DataFrame(ham_text,columns=['text']) #將list改成dataframe
ham_data['label']='0' #將資料類別加上 0:ham 1:spam
ham_data.head()


,text,label
0,這事兒還有續集 因為我姐她們買房子後來裝修的時候沒錢她婆婆他們就贊助瞭萬塊 這下一吵她婆婆就...,0
1,越發覺得這個姐夫不錯瞭 那萬還是趕緊還瞭吧 標題姐姐快要生孩子瞭很感慨 發信站水木社區站內 ...,0
2,或許某些農村吧 在一個當性彆勞動力富裕程度的地方還是可以理解的 標題姐姐快要生孩子瞭很感慨 ...,0
3,最近經常看到你說一些很武斷的話 所謂保胎確實是要躺在床上不能動的尤其是先兆流産有齣血的時候 ...,0
4,他們分手是說性格不閤 沒多久然後我們交往 還算比較愉快 當初因為他經常和聯係我曾跟他吵過多次...,0


In [114]:
for fileName in listspam:
        with open('data/spam/'+fileName, 'r',encoding='gbk') as f:
            text=[]
            # 过滤掉非中文字符
            for line in f.readlines():
                pattern = re.compile('[^\u4e00-\u9fa5]')
                line = pattern.sub("", line)
                content = line.strip().split()
                text = text+content
            text = " ".join(text)
            spam_text.append(HanziConv.toTraditional(text))
spam_data = pd.DataFrame(spam_text,columns=['text'])
spam_data['label']='1'
spam_data.head()

,text,label
0,會員注冊邀請書 全國最低價域名空間大放送 國際頂級域名注冊驚暴價元年元年連續注冊年以上 國傢...,1
1,你好 以茶會友以茶聯誼喝茶就喝安溪鐵觀音 本廠是專業生産批發各等級鐵觀音茶葉的購買聯係 在綫...,1
2,尊敬的先生小姐 您好 首先感謝您抽時間查看以下信息 我公司一直緻力於研發生産銷售施工環氧樹脂...,1
3,先生 你好 批發麻醉藥催情藥墮胎藥避孕藥飲料型噴霧型香煙型揮發型等各類國際國內製藥企業推齣的...,1
4,貴公司負責人經理財務您好 我是深圳市創偉實業有限公司的我司實力雄厚有著良 好的社會關係也有部...,1


In [115]:
all_data = pd.concat([spam_data,ham_data],axis=0, ignore_index=True)  #將ham lsit與spam list串起來
all_data = all_data.sample(frac=1).reset_index(drop=True) #sample frac = 1 是把數據都打亂 resr_index是把index也重設
print('data shape：',all_data.shape)
print('spams in rows：',all_data.loc[all_data['label']=="1"].shape[0])
print('hams in rows：',all_data.loc[all_data['label']=="0"].shape[0])
all_data.head(6)

data shape： (14838, 2)
spams in rows： 7775
hams in rows： 7063


,text,label
0,耶 園華清 修改於修改本文 來源水木社區,0
1,軟件功能包括基本資料進倉管理銷售管理統計管理其他收入支齣管理係統維護 基本資料商品資料員工資...,1
2,貴公司領導 您好我公司現有抵帳新叉車颱裝載機颱因我公司閑置無用現以優惠價格市場半價齣售如貴公...,1
3,會進入檔案的吧 不太好 標題急問關於違約金問題 發信站水木社區站內 待遇確實有點低衝動瞭點 ...,0
4,你好 你想讓你的手機同時擁有兩個號碼嗎 我們的兄弟連手機雙卡幫你實現這一願望讓你花最少的錢達...,1
5,東方少年歌 唧唧復唧唧中華長嘆息問君何所思問君何所憶中華獨愴然 諸君聽仔細東鄰有惡魔擾我百年...,0


In [116]:
#載入 停用詞 (stopwords) 自動過濾掉某些字或詞
stopwords = codecs.open(os.path.join(SaveDirectory+'\data', 'stopwords_tr.txt'), 'r', 'utf-8').read().split('\r\n')

In [117]:
processed_texts = []
for text in all_data["text"]:
    words = []
    seg_list = jieba.cut(text) #利用Jieba斷詞
    for seg in seg_list: 
        # isalpha()檢測詞是否由字符組成，且不在停用詞的list中
        if (seg.isalpha()) & (seg not in stopwords):
            words.append(seg)
    sentence = " ".join(words)
    processed_texts.append(sentence)
all_data["text"] = processed_texts #利用過濾且斷好的詞取代文本
all_data.head(6)

,text,label
0,耶 園華清 修改 修改 本文 來源 社區,0
1,軟件 功能 包括 資料 進倉 管理 銷售 管理 統計 管理 收入 支 齣 管理 係 統維護 ...,1
2,公司 領導 您好 公司 現有 抵帳 新叉車 颱 裝載機 颱 公司 閑 置 無用 現以 優惠價...,1
3,進入 檔案 不太好 標題 急 問關 違約金 問題 發信 站 社區 站 待遇 確實 有點 低 ...,0
4,你好 想 手機 擁有 兩個 號碼 兄弟 連手 機雙卡 幫 實現 這一願望 花 最少 錢 達 ...,1
5,東方 少年 歌 唧唧 復 唧唧 中華長 嘆息 問君何 所思 問君何 所憶 中華 獨愴 諸君 ...,0


In [118]:
#traindate & testdata
train_set, test_set, trainlabel, testlabel = train_test_split(all_data["text"], all_data["label"], test_size = 0.15)

In [119]:
print("train:"+str(len(train_set)))
print("test:"+str(len(test_set)))
trainlabel[trainlabel=='1'].shape

train:12612
test:2226


(6625,)

In [120]:
trainlabel_orig = trainlabel_.copy()
# 取消一些標記的數據
unlabeled_size = 2597
#loc可以選擇數據(把上面數量的POSTIVE DATA隱藏為UNLABELED DATA)
trainlabel.loc[
    np.random.choice(
        trainlabel[trainlabel == '1'].index, 
        replace = False, 
        size = unlabeled_size
    )
] = '0'
trainlabel[trainlabel=='1'].shape

(4028,)

In [121]:
positive_train = trainlabel[trainlabel=='1'].index
positive_data = train_set[positive_train]
positive_spam = pd.DataFrame({'text':positive_data, 'label':trainlabel[positive_train]})
positive_spam.head()

,label,text
1464,1,網絡 營銷 大全 光盤 專業版 廣告 信息 直接 投入 客戶 信箱 員 製 網 站 一次 付...
1189,1,你好 騰達 實業 有限公司 因進 項 每月 部分 結餘 發票 優惠 外代 開 普通 發票 國...
4999,1,本郵件 屬群 發性質 自動 發送 打擾 請 諒並 隨手 刪掉 桂豐 牌車 位鎖 分手 動遙 ...
11136,1,電郵 地址 庫 電郵 地址 庫 公司 目前 國內 最大 電子 郵件 地址 專業供 應商 嚮 ...
10902,1,公司 負責人 經理 財務 您好 華龍 公司 受多 傢 公司 委托 嚮 外低 點代開 部分 增...


In [122]:
import re
def generate_key_list(all_data, size_table=200,ignore=2):
    dict_spam_raw = dict()
    dict_genuine_raw = dict()
    dict_IDF = dict()

    # 去除字母外的所有內容.
    for i in range(all_data.shape[0]):
        finds = all_data.iloc[i].text.split(" ")
        if all_data.iloc[i].label == '1':
            for find in finds:
                if len(find)<ignore: continue
                try:
                    dict_spam_raw[find] = dict_spam_raw[find] + 1
                except:	
                    dict_spam_raw[find] = dict_spam_raw.get(find,1) #若是新的文字因為後面get找不到這個索引會返回逗點後面的值
                    dict_genuine_raw[find] = dict_genuine_raw.get(find,0)
        else:
            for find in finds:
                if len(find)<ignore: continue
                try:
                    dict_genuine_raw[find] = dict_genuine_raw[find] + 1
                except:	
                    dict_genuine_raw[find] = dict_genuine_raw.get(find,1)
                    dict_spam_raw[find] = dict_spam_raw.get(find,0)

        word_set = set()
        for find in finds:
            if not(find in word_set):
                if len(find)<ignore: continue
                try:
                    dict_IDF[find] = dict_IDF[find] + 1
                except:	
                    dict_IDF[find] = dict_IDF.get(find,1)
            word_set.add(find)
    word_df = pd.DataFrame(list(zip(dict_genuine_raw.keys(),dict_genuine_raw.values(),dict_spam_raw.values(),dict_IDF.values())))
    word_df.columns = ['keyword','genuine','spam','IDF']
    word_df['genuine'] = word_df['genuine'].astype('float')/all_data[all_data['label']=='0'].shape[0]
    word_df['spam'] = word_df['spam'].astype('float')/all_data[all_data['label']=='1'].shape[0]
    word_df['IDF'] = np.log10(word_df.shape[0]/word_df['IDF'].astype('float'))
    word_df['genuine_IDF'] = word_df['genuine']*word_df['IDF']
    word_df['spam_IDF'] = word_df['spam']*word_df['IDF']
    word_df['diff']=word_df['spam_IDF']-word_df['genuine_IDF']
    selected_spam_key = word_df.sort_values('diff',ascending=False)  
    keyword_dict = dict()
    i = 0
    for word in selected_spam_key.head(size_table).keyword:
        keyword_dict.update({word.strip():i})
        i+=1
    return keyword_dict   
# build a tabu list based on the training data
size_table = 200               # 多少特徵維度去分類SPAM
word_len_ignored = 2            # 忽略那些比這個還要小的字詞
keyword_dict=generate_key_list(positive_spam, size_table, word_len_ignored)

In [123]:
#排序越前面的代表出現在SPAM的機率越大
for key,value in keyword_dict.items():
    print('{key}:{value}'.format(key = key, value = value))

網絡:0
營銷:1
大全:2
光盤:3
專業版:4
廣告:5
信息:6
直接:7
投入:8
客戶:9
信箱:10
一次:11
付費:12
永久:13
享受:14
免費會員:15
升級:16
服務:17
每月:18
提供:19
以上:20
免費:21
更新:22
會員:23
搜索:24
答疑:25
解答:26
使用:27
各類:28
問題:29
包括:30
萬綜:31
郵件:32
地址:33
萬分:34
地區:35
分行:36
業郵箱:37
自由:38
選擇:39
群發:40
贈送:41
軟件:42
郵箱:43
發布:44
切割:45
過濾重:46
注冊:47
文件:48
物超所值:49
詳細:50
圖解:51
說明:52
簡單:53
易懂:54
北京:55
易友:56
商務:57
電話:58
留言:59
鄭權友:60
張敏:61
北京市:62
西四:63
中路:64
你好:65
騰達:66
實業:67
有限公司:68
因進:69
部分:70
結餘:71
發票:72
優惠:73
外代:74
普通:75
國稅:76
地稅:77
代開:78
範圍:79
商品:80
銷售:81
運輸:82
餐飲:83
建築安裝:84
服務發:85
數量:86
額度:87
大小:88
衡量:89
點數:90
稅率:91
左右:92
增值:93
公司:94
鄭重:95
承諾:96
所用:97
票據:98
單位:99
稅務:100
局所:101
申領:102
可上:103
網查詢:104
稅務局:105
抵扣:106
驗證:107
電話劉:108
先生:109
李小姐:110
本郵件:111
屬群:112
發性質:113
自動:114
發送:115
打擾:116
諒並:117
隨手:118
刪掉:119
桂豐:120
牌車:121
位鎖:122
分手:123
動遙:124
控電動:125
兩種:126
具有:127
操作:128
方便:129
美觀:130
耐用:131
安裝:132
簡便:133
特點:134
小區:135
停車場:136
專用:137
車位:138
理想:139
品能:140
防止:141
占用:142
保證:143
愛車:144
保留:145
一個屬:146
空間:147
桂豐鎖:148
車器:149
采用:150
優質:151
鋼材:152
開模衝:153
將車輛:154

In [126]:
unlabeled_train = trainlabel[trainlabel=='0'].index
unlabeled_data = train_set[unlabeled_train]
unlabeled_df = pd.DataFrame({'text':unlabeled_data, 'label':trainlabel[unlabeled_train]})


In [134]:
all_data = pd.concat([positive_spam,unlabeled_df],axis=0, ignore_index=True)  #將ham lsit與spam list串起來
all_data = all_data.sample(frac=1) #sample frac = 1 是把數據都打亂
all_train = all_data.index

In [128]:
def convert_Content(text, keyword_dict):
    #判斷是否有此特徵
    m = len(keyword_dict) #維度數量
    res = np.int_(np.zeros(m)) #建置一個幾維度的向量
    finds = text.split(" ") #將文本內容切割(類似中文斷詞)
    for find in finds:
        try:
            #若比對完有此特徵則特徵改為1
            i = keyword_dict[find] 
            res[i]=1
        except:
            continue
    return res
def raw2feature(all_data,keyword_dict):
    n_all_data = all_data.shape[0]
    m = len(keyword_dict)
    X_all_data = np.zeros((n_all_data,m));
    Y_all_data = np.int_(all_data.label=='1')
    for i in range(n_all_data):
        X_all_data[i,:] = convert_Content(all_data.iloc[i].text, keyword_dict)
    return [X_all_data,Y_all_data]
     
all_data_matrix=raw2feature(all_data,keyword_dict)


In [129]:
# positive_train  正樣本的index
# unlabeled_train 未標記樣本的index
# all_train 混合完的index